# Autonomous Perception Robustness Testing Framework (APRTF)
### Development Journal

We show that our general framework can be used on the [NuScenes](https://www.nuscenes.org/) dataset using a multi-stage analysis proposed in ["Perception robustness testing at different levels of generality"](https://www.journalfieldrobotics.org/FR/Papers_files/10_Pezzementi.pdf).

In [1]:
import os
import torch
import torchvision.transforms as T

from nuscenes.utils.geometry_utils import view_points
from nuscenes import NuScenes
data_dir = './data/sets/nuScenes'
nusc = NuScenes(version='v1.0-mini', dataroot=data_dir, verbose=True)

# torchvision reference code
from aprtf.dataset import PedestrianDetectionDataset, get_transform
from aprtf.references import utils
from aprtf.visuals import visualize_results

print("All packages imported!")


SEED = 42
torch.manual_seed(SEED)

print('Random Seed Set!')

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.425 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
All packages imported!
Random Seed Set!


## I. Pedestrian Detection

### Data and Labels

Time-ordered iterator of images and bounding boxes.

In [2]:
# nuScenes
def box2bb(box, cam_intrinsic):
    corners = torch.tensor(view_points(box.corners(), view=cam_intrinsic, normalize=True)[:2, :])
    bb = torch.cat([torch.min(corners, dim=1).values, torch.max(corners, dim=1).values]).tolist()
    return bb
    
category = 'pedestrian'
sensor = 'CAM_FRONT'
visibility_threshold = 2

odgt = []

for scene in nusc.scene:
    next_sample_token = scene['first_sample_token']
    while next_sample_token:
        sample = nusc.get('sample', next_sample_token)
        sample_data = nusc.get('sample_data', sample['data'][sensor])

        # image filepaths
        sample_data_fp = os.path.join(data_dir,sample_data['filename'])

        # bounding boxes
        sample_data_bbs = []
        for ann in sample['anns']:
            _, box, cam_intrinsic = nusc.get_sample_data(sample['data'][sensor], selected_anntokens=[ann])
            if len(box) > 1:
                raise ValueError('more than one annotation')

            visibility_token = nusc.get('sample_annotation', ann)['visibility_token']
            visibility = int(visibility_token)
            if (len(box) == 1) and (category in box[0].name) and (visibility >= visibility_threshold):
                bb = box2bb(box[0], cam_intrinsic)
                sample_data_bbs.append(bb)

        # odgt
        odgt.append(
            {
                'image': sample_data_fp,
                'annotations': sample_data_bbs
            }
        )

        # next sample
        next_sample_token = sample['next']

dataset = PedestrianDetectionDataset(odgt, get_transform(train=False))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=1, shuffle=False, num_workers=1,
 collate_fn=utils.collate_fn)


images = []
gt_bbs = []
dt_bbs = [] 
for i in range(9):
    output_ims, targets = next(iter(data_loader))
    target = [{k: v for k, v in t.items()} for t in targets]

    image = T.ToPILImage()(output_ims[0])
    images.append(image)

    gt_bbs.append(target[0]['boxes'])
    dt_bbs.append(target[0]['boxes'] * 0.97)
#visualize_results('output.png', images, gt_bbs, dt_bbs)

### Metric

Recall and $FPR_A$.

In [3]:
def recall():
    pass

def FPRa():
    pass